In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg


Import Data

In [2]:
# Load and preprocess data
mbp_csv = pd.read_csv('xnas-itch-20240822.mbp-1.csv', parse_dates=['ts_recv', 'ts_event'])

# Calculate the log-arithmetic middle price
#mbp_csv['log_arith_middle_price'] = (np.log(mbp_csv['bid_px_00']) + np.log(mbp_csv['ask_px_00'])) / 2

mbp_csv['log_arith_middle_price'] = np.log((mbp_csv['bid_px_00']+mbp_csv['ask_px_00'])/2)

In [3]:
mbp_csv

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,...,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,log_arith_middle_price
0,2024-08-22 13:00:00.148489656+00:00,2024-08-22 13:00:00.148324259+00:00,1,2,11667,C,A,0,129.93,600,...,165397,11891132,129.80,129.93,899,306,4,3,NVDA,4.866495
1,2024-08-22 13:00:01.015421988+00:00,2024-08-22 13:00:01.015255234+00:00,1,2,11667,T,N,0,129.89,700,...,166754,11895306,129.80,129.93,899,306,4,3,NVDA,4.866495
2,2024-08-22 13:00:06.026216498+00:00,2024-08-22 13:00:06.026050629+00:00,1,2,11667,A,A,0,129.91,4,...,165869,11908757,129.80,129.91,899,4,4,1,NVDA,4.866418
3,2024-08-22 13:00:08.021418483+00:00,2024-08-22 13:00:08.021252273+00:00,1,2,11667,A,B,0,129.81,100,...,166210,11913875,129.81,129.91,100,4,1,1,NVDA,4.866457
4,2024-08-22 13:00:08.378809454+00:00,2024-08-22 13:00:08.378639226+00:00,1,2,11667,T,N,0,129.89,100,...,170228,11914309,129.81,129.91,100,4,1,1,NVDA,4.866457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387422,2024-08-22 20:59:46.873416500+00:00,2024-08-22 20:59:46.873250383+00:00,1,2,11667,A,B,0,124.33,1,...,166117,631379773,124.33,124.42,1,100,1,1,NVDA,4.823301
3387423,2024-08-22 20:59:50.256406269+00:00,2024-08-22 20:59:50.256237949+00:00,1,2,11667,T,A,0,124.33,1,...,168320,631381505,124.33,124.42,1,100,1,1,NVDA,4.823301
3387424,2024-08-22 20:59:50.256406269+00:00,2024-08-22 20:59:50.256237949+00:00,1,2,11667,T,N,0,124.33,49,...,168320,631381506,124.30,124.42,826,100,5,1,NVDA,4.823181
3387425,2024-08-22 20:59:50.363312798+00:00,2024-08-22 20:59:50.363146531+00:00,1,2,11667,A,B,0,124.32,2,...,166267,631381570,124.32,124.42,2,100,1,1,NVDA,4.823261


Helper Functions 

In [4]:
# Function to compute subsampled variance [X, X]^K_T
def compute_subsampled_variance(log_returns, K):
    n = len(log_returns)
    subsampled_variance = 0
    
    log_returns = list(log_returns)
    
    for i in range(n - K):
        pt1 = log_returns[i + K]
        pt2 = log_returns[i]
        subsampled_variance += (pt1 - pt2)**2
        
    subsampled_variance /= K
    return subsampled_variance

In [5]:
# Function to compute full sample variance [X, X]^{All}_T
def compute_full_sample_variance(log_returns):
    return np.sum(np.diff(log_returns)**2)

In [6]:
# Function to compute TSRV
def compute_TSRV(log_returns, K):
    n = len(log_returns)
    subsampled_variance = compute_subsampled_variance(log_returns, K)
    full_sample_variance = compute_full_sample_variance(log_returns)
    z = (n - K + 1) / K
    tsrv = (1 - z/n)**(-1) * (subsampled_variance - (z/n) * full_sample_variance)
    return tsrv

In [7]:
def compute_realized_volatility(df, symbol):
    df_symbol = df[df['symbol'] == symbol].set_index('ts_event')
    
    # Calculate for 10 min
    df_10min = df_symbol.resample('10T')
    tsrvs_10min = []
    timestamps = []
    for name, group in df_10min:
        log_returns = group['log_arith_middle_price'].values
        tsrv = compute_TSRV(log_returns, K=20)
        tsrvs_10min.append(np.sqrt(tsrv))
        timestamps.append(name)
    
    # Create a DataFrame with timestamps and 10-min volatilities
    vol_df = pd.DataFrame({'timestamp': timestamps, '10min_vol': tsrvs_10min})
    vol_df.set_index('timestamp', inplace=True)
    
    # Calculate 30-min volatility
    vol_df['30min_vol'] = np.sqrt((vol_df['10min_vol']**2 + 
                                   vol_df['10min_vol'].shift(1)**2 + 
                                   vol_df['10min_vol'].shift(2)**2) / 3)
    
    # Calculate 1-hour volatility
    vol_df['1hour_vol'] = np.sqrt((vol_df['10min_vol']**2 + 
                                   vol_df['10min_vol'].shift(1)**2 + 
                                   vol_df['10min_vol'].shift(2)**2 + 
                                   vol_df['10min_vol'].shift(3)**2 + 
                                   vol_df['10min_vol'].shift(4)**2 + 
                                   vol_df['10min_vol'].shift(5)**2) / 6)
    
    return vol_df


Calculate Realised Volatility 

In [8]:
# Compute realized volatility for NVDA and CAKE
nvda_vol = compute_realized_volatility(mbp_csv, 'NVDA')
cake_vol = compute_realized_volatility(mbp_csv, 'CAKE')

print(nvda_vol)

/tmp/ipykernel_11926/1921819226.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_10min = df_symbol.resample('10T')
/tmp/ipykernel_11926/1921819226.py:11: RuntimeWarning: invalid value encountered in sqrt
  tsrvs_10min.append(np.sqrt(tsrv))


                           10min_vol  30min_vol  1hour_vol
timestamp                                                 
2024-08-22 13:00:00+00:00   0.001600        NaN        NaN
2024-08-22 13:10:00+00:00   0.001205        NaN        NaN
2024-08-22 13:20:00+00:00   0.003660   0.002409        NaN
2024-08-22 13:30:00+00:00   0.007781   0.005013        NaN
2024-08-22 13:40:00+00:00   0.006237   0.006133        NaN
2024-08-22 13:50:00+00:00   0.004732   0.006373   0.004817
2024-08-22 14:00:00+00:00   0.004900   0.005332   0.005175
2024-08-22 14:10:00+00:00   0.004839   0.004824   0.005518
2024-08-22 14:20:00+00:00   0.003947   0.004583   0.005550
2024-08-22 14:30:00+00:00   0.003594   0.004160   0.004782
2024-08-22 14:40:00+00:00   0.003325   0.003631   0.004270
2024-08-22 14:50:00+00:00   0.003250   0.003393   0.004032
2024-08-22 15:00:00+00:00   0.003196   0.003257   0.003736
2024-08-22 15:10:00+00:00   0.003187   0.003211   0.003427
2024-08-22 15:20:00+00:00   0.003366   0.003251   0.0033

/tmp/ipykernel_11926/1921819226.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_10min = df_symbol.resample('10T')


In [9]:

print(cake_vol)

                           10min_vol  30min_vol  1hour_vol
timestamp                                                 
2024-08-22 13:20:00+00:00   0.002183        NaN        NaN
2024-08-22 13:30:00+00:00   0.003236        NaN        NaN
2024-08-22 13:40:00+00:00   0.004422   0.003405        NaN
2024-08-22 13:50:00+00:00   0.002286   0.003428        NaN
2024-08-22 14:00:00+00:00   0.002446   0.003202        NaN
2024-08-22 14:10:00+00:00   0.002782   0.002513   0.002993
2024-08-22 14:20:00+00:00   0.001678   0.002348   0.002938
2024-08-22 14:30:00+00:00   0.002206   0.002267   0.002774
2024-08-22 14:40:00+00:00   0.002589   0.002190   0.002357
2024-08-22 14:50:00+00:00   0.002205   0.002340   0.002344
2024-08-22 15:00:00+00:00   0.002863   0.002566   0.002421
2024-08-22 15:10:00+00:00   0.001885   0.002353   0.002273
2024-08-22 15:20:00+00:00   0.001453   0.002149   0.002247
2024-08-22 15:30:00+00:00   0.001466   0.001614   0.002144
2024-08-22 15:40:00+00:00   0.002185   0.001735   0.0020

In [10]:
def perform_regression_analysis(data, symbol):
    print(f"\nRegression Analysis for {symbol}")

    data['10min_vol_next'] = data['10min_vol'].shift(-1)  # Target variable
    data.dropna(inplace=True)
    
    # Split the sample (2/3 for training, 1/3 for testing)
    split_point = int(len(data) * 2/3)
    train_data = data.iloc[:split_point]
    test_data = data.iloc[split_point:]
    
    # Prepare training data
    y_train = train_data['10min_vol_next']
    X_train = train_data[['10min_vol', '30min_vol', '1hour_vol']]
    X_train = sm.add_constant(X_train)
    
    # Fit the model
    model = sm.OLS(y_train, X_train).fit()
    print(model.summary())
    
    # Out-of-sample forecast
    X_test = test_data[['10min_vol', '30min_vol', '1hour_vol']]
    X_test = sm.add_constant(X_test)
    y_test = test_data['10min_vol_next']
    
    y_pred = model.predict(X_test)
    mse = np.mean((y_test - y_pred)**2)
    print(f"Out-of-sample forecast MSE: {mse}")
    
    # AutoReg model (replacing AR) for comparison
    ar_model = AutoReg(train_data['10min_vol'], lags=6).fit()
    ar_predictions = ar_model.forecast(steps=len(test_data))
    ar_mse = np.mean((test_data['10min_vol'] - ar_predictions)**2)
    print(f"AutoReg(6) model MSE: {ar_mse}")
    
    relative_performance = (ar_mse - mse) / ar_mse * 100
    print(f"Relative improvement over AutoReg(6): {relative_performance:.2f}%")

In [11]:
perform_regression_analysis(nvda_vol, 'NVDA')
perform_regression_analysis(cake_vol, 'CAKE')


Regression Analysis for NVDA
                            OLS Regression Results                            
Dep. Variable:         10min_vol_next   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     2.493
Date:                Sun, 08 Sep 2024   Prob (F-statistic):             0.0880
Time:                        18:11:46   Log-Likelihood:                 146.02
No. Observations:                  25   AIC:                            -284.0
Df Residuals:                      21   BIC:                            -279.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0032 

/home/sasquach/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
/home/sasquach/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/home/sasquach/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
/home/sasquach/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a